# Keyword, Keysentence 추출

In [ ]:
from IPython.display import clear_output

In [ ]:
# 토크나이저 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

%cd /content
clear_output()

In [ ]:
# textrank 설치
!git clone --recursive https://github.com/lovit/textrank
import sys
sys.path.insert(0,'/content/textrank')
clear_output()

In [ ]:
from textrank import KeywordSummarizer
from textrank import KeysentenceSummarizer
from konlpy.tag import Mecab

In [ ]:
rawtext = '근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다. \
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다. \
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다. 아침 식사를 거르지 \
않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 \
체내 노폐물을 배출하고 혈압을 낮춰준다. 운동은 하루에 10분 정도만 하는 게 좋으며 \
운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다. \
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 \
피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.'

## 명사 토크나이징

In [ ]:
# pos 태깅 NN(명사) 단어 추출
def pos_nn_words(sent):
    mecab = Mecab()
    words = mecab.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w and list(w)[1]!='/')]
    
    words = [w.split('/')[0] for w in words]
    return words

In [ ]:
mecab = Mecab()
print(mecab.nouns(rawtext))  # mecab 기본 제공 nouns 메서드
print(pos_nn_words(rawtext)) # 위에서 정의한 pos_nn_words 함수 ==>> mecab.pos로 pos태깅하고, NN 태그만 추출

# 결론.. pos_nn_words 사용해야됌. 왜냐면 mecab.nouns는 '무엇', '것', '뿐' 같은 명사도 추출함

['근육', '무엇', '규칙', '생활', '습관', '중요', '아침', '식사', '단백질', '비타민', '과일', '채소', '섭취', '것', '하루', '분', '이상', '수면', '것', '도움', '아침', '식사', '규칙', '운동', '혈액', '순환', '도움', '뿐', '신진대사', '촉진', '체내', '노폐물', '배출', '혈압', '운동', '하루', '분', '정도', '게', '운동', '후', '스트레칭', '근육', '량', '운동', '후', '잠자리', '것', '아침', '몸', '피곤', '때문', '무리', '역효과', '수']
['근육', '규칙', '생활', '습관', '중요', '아침', '식사', '단백질', '비타민', '과일', '채소', '섭취', '하루', '이상', '수면', '도움', '아침', '식사', '규칙', '운동', '혈액', '순환', '도움', '신진대사', '촉진', '체내', '노폐물', '배출', '혈압', '운동', '하루', '정도', '운동', '스트레칭', '근육', '운동', '잠자리', '아침', '피곤', '때문', '무리', '역효과']


## 빈도 기반으로 키워드 추출

In [ ]:
from collections import Counter

def keywords_by_freq(text,n):
    # keywords = list(filter(lambda x: len(x)>1, word_list_1)) ## 2글자 이상 단어만 고려
    count = Counter(text)
    
    most_common = count.most_common(n)
    keywords = []
    keywords_cnt = []
    for key, cnt in most_common:
        keywords.append(key)
        keywords_cnt.append(cnt)
        
    return keywords, keywords_cnt

In [ ]:
nn_words = pos_nn_words(rawtext)
keyword_prob = keywords_by_freq(nn_words, 5)

In [ ]:
print("------------------------")
print("  빈도로 추출한 키워드  ")
print("------------------------")
for i in range(len(keyword_prob[0])):
    print(f'{i+1}위: {keyword_prob[0][i]} ({keyword_prob[1][i]})')

------------------------
  빈도로 추출한 키워드  
------------------------
1위: 운동 (4)
2위: 아침 (3)
3위: 근육 (2)
4위: 규칙 (2)
5위: 식사 (2)


In [ ]:
### 빈도로 추출하는 키워드도 의미가 있긴 한데, 더 분석적인 방법?이 textrank 알고리즘을 기반으로 키워드 추출하는 거다.
    
    ### 어떤 면에서 더 좋은지 아직 잘 몰라서 찾아봐야됌..

## textrank 기반 키워드 추출

In [ ]:
nn_words = pos_nn_words(rawtext)

In [ ]:
def summarizer_text(texts, n):
    summarizer = KeysentenceSummarizer(
        tokenize = pos_nn_words,
        min_sim = 0.3,
        verbose = False
    )
    sent = []
    keysents = summarizer.summarize(texts, topk=n)

    for _, _, a in keysents:
        sent.append(a)
    
    final_sent = []
    
    for i in texts:
        if i in sent:
            final_sent.append(i)
    
    return final_sent

In [ ]:
def keywords_by_textrank(text, n):
    keyword_extractor = KeywordSummarizer(
    tokenize = pos_nn_words,
    window = -1,
    verbose = False,
    min_count=1) # FOR working with extremely short text.

    textrank_keywords = keyword_extractor.summarize([text], n)

    return textrank_keywords

In [ ]:
textrank_keywords = keywords_by_textrank(rawtext, 5)
textrank_keywords

[('운동', 2.7467144563918784),
 ('아침', 2.096774193548387),
 ('근육', 1.4468339307049),
 ('규칙', 1.4468339307049),
 ('식사', 1.4468339307049)]

In [ ]:
exam =  '천재는 1의 영감과 99의 목표의식을 가지고서 모든 일에 열중하고 있으며 하루하루 자아향상에 노력해 왔습니다.'
keywords_by_textrank(exam, 3)

[('노력', 0.9999999999999997),
 ('향상', 0.9999999999999997),
 ('자아', 0.9999999999999997)]

In [ ]:
[x[0] for x in textrank_keywords], [x[1]]

['운동', '아침', '근육', '규칙', '식사']

In [ ]:
print("------------------------")
print("  Textrank로 추출한 키워드  ")
print("------------------------")
for i, keywords in enumerate(textrank_keywords):
    print(f"{i+1}위: {keywords[0]} ({round(keywords[1], 2)})")

------------------------
  Textrank로 추출한 키워드  
------------------------
1위: 운동 (1.55)
2위: 아침 (1.2)
3위: 도움 (0.85)
4위: 하루 (0.85)
5위: 식사 (0.85)


## textrank 키워드가 포함된 문장 추출

In [ ]:
# input text should be 'textrank_keywords' and 'string'

def keyword_dict(keyword, text):
    dic = {k[0]: [] for k in keyword}

    for k in dic:
        for txt in text.split('.'):
            if k in txt:
                dic[k].append(txt)

    return dic

In [ ]:
keyword_with_sentences = keyword_dict(textrank_keywords, rawtext)
keyword_with_sentences # 이렇게 반환된 dictionary는 textrank score 순서대로 나열된 게 아님

{'도움': [' 또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다',
  ' 아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다'],
 '식사': [' 특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다',
  ' 아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다'],
 '아침': [' 특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다',
  ' 아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다',
  ' 운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다'],
 '운동': [' 아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다',
  ' 운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다',
  ' 운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다'],
 '하루': [' 또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다',
  ' 운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다']}

In [ ]:
keyword_sent1 = keyword_with_sentences[textrank_keywords[0][0]]
keyword_sent2 = keyword_with_sentences[textrank_keywords[1][0]]
keyword_sent3 = keyword_with_sentences[textrank_keywords[2][0]]

In [ ]:
keyword_sent1 # 운동

[' 아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다',
 ' 운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다',
 ' 운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다']

In [ ]:
keyword_sent2 # 아침

[' 특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다',
 ' 아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다',
 ' 운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다']

In [ ]:
keyword_sent3 # 도움

[' 또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다',
 ' 아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다']

In [ ]:
# visualization
for keyword in textrank_keywords:
    print('-'*80)
    print(f'키워드 "{keyword[0]}"이/가 포함된 문장')
    print('-'*80)
    for sent in keyword_with_sentences[keyword[0]]:
        print('o',sent)
        print()
    print()

--------------------------------------------------------------------------------
키워드 "운동"이/가 포함된 문장
--------------------------------------------------------------------------------
o  아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다

o  운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다

o  운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다


--------------------------------------------------------------------------------
키워드 "아침"이/가 포함된 문장
--------------------------------------------------------------------------------
o  특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다

o  아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다

o  운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다


--------------------------------------------------------------------------------
키워드 "도움"이/가 포함된 문장
--------------------------------------------------------------------------------
o  또한 하

## 키워드 포함된 문장 counter
- 위 리스트를 보면.. 중복된 문장, 즉 중요한 키워드를 여럿 포함하고 있는 문장들이 있다.
- 키워드가 가장 많이 들어있는 문장이 뭔지 보여주기!

In [ ]:
sents_count = []
for key in keyword_with_sentences.keys():
    sents_count += keyword_with_sentences[key]

In [ ]:
sents_count = Counter(sents_count)
sents_count

Counter({' 또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다': 2,
         ' 아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다': 4,
         ' 운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다': 2,
         ' 운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다': 2,
         ' 특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다': 2})

In [ ]:
# 가장 키워드가 많이 들어간 문장 2개 추출
sents_count.most_common(2)

[(' 아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다',
  4),
 (' 운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다', 2)]

In [ ]:
# 이쯤에서 원본 문장 다시 확인
rawtext

'근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다. 특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다. 또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다. 아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다. 운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다. 운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.'